In [1]:
from collections import defaultdict
import numpy as np
import pandas as pd
import scipy.io as io
import gzip
import math
import os
import re
import scipy
from utils import *

In [ ]:
#TODO: still need to understand why those lines in Maria's code are necessary

zurich['BNCfreq'] = zurich.WORDstrip.map(lambda x: unigrdict.get(str(x).lower()))
zurich.BNCfreq = zurich.BNCfreq.fillna(zurich.BNCfreq.min())
zurich.BNCfreq = zurich.BNCfreq/100 #because 100 million word - to get freq per million
zurich.BNCfreq = np.log(zurich.BNCfreq)

zurich['BNCfreqinv']= -zurich.BNCfreq

In [ ]:
def mk_dataframe(task:str, subject:int, level:str, scaling:str):
    """
        Args: Task number ("task1", "task2", "task3" , test subject(0-11).
        Return: DataFrame on word level.
    """
    bnc_freq = get_bncfreq()
    files = get_matfiles(task)
    data = io.loadmat(files[subject], squeeze_me=True, struct_as_record=False)['sentenceData']
    
    if level == 'sentence':
        fields = ['SentLen', 'omissionRate', 'nFixations', 'meanPupilSize', 'GD', 'TRT', 'FFD', 'SFD', 
                  'GPT', 'BNCFreq']
        features = np.zeros((len(data), len(fields)))
        
    elif level == 'word':
        n_words = sum([len(sent.word) for sent in data])
        fields = ['Sent_ID', 'Word_ID', 'Word', 'nFixations', 'meanPupilSize', 
                  'GD', 'TRT', 'FFD', 'SFD', 'GPT', 'WordLen', 'BNCFreq']
        df = pd.DataFrame(index=range(n_words), columns=[fields])
        k = 0
    else:
        raise Exception('Data can only be processed on sentence or word level')
        
    for i, sent in enumerate(data):
        print('Iter:', i)
        print()
        print([word.content for word in sent.word])
        print()
        for j, word in enumerate(sent.word):
            token = re.sub('[^\w\s]', '', word.content)
            token = token.lower() if j == 0 else token
            if level == 'sentence':
                features[i,2:-1] += [getattr(word, field) if hasattr(word, field)\
                                    and not isinstance(getattr(word, field), np.ndarray) else\
                                    0 for field in fields[2:-1]]
                #TODO: figure out whether divsion by 100 leads to -inf values after log computation 
                features[i,-1] += np.log(bnc_freq[token]/100) if bnc_freq[token]/100 != 0 else 0
            else:
                df.iloc[k, 0] = str(i) + '_NR' if task=='task1' or task=='task2' else str(i) + '_TSR'
                df.iloc[k, 1] = j
                df.iloc[k, 2] = token
                df.iloc[k, 3:-2] = [getattr(word, field) if hasattr(word, field)\
                                    and not isinstance(getattr(word, field), np.ndarray) else\
                                    0 for field in fields[3:-2]]
                df.iloc[k, -2] = len(token)
                #TODO: figure out whether divsion by 100 leads to -inf values after log computation
                df.iloc[k,-1] = np.log(bnc_freq[token]/100) if bnc_freq[token]/100 != 0 else 0
                k += 1
                
        if level == 'sentence':
            features[i, 0] = len(sent.word)
            features[i, 1] = sent.omissionRate
            features[i, 2:] /= len(sent.word)
    
    #handle -inf, inf and NaN values
    #if level == 'sentence': 
    #    features = inf_check(features)
    if level == 'word':
        df.iloc[:,:-1].fillna(0, inplace=True)
        df.iloc[:,-1].fillna(getattr(df,fields[-1]).values.min(), inplace=True)  
        df.replace([np.inf, -np.inf], np.nan).dropna(axis=0, inplace=True)

    #normalize data featurewise
    if scaling == 'min-max':
        if level == 'sentence':
            features = np.array([(feat - min(feat))/(max(feat) - min(feat)) for feat in features.T])
            df = pd.DataFrame(data=features.T, index=range(features.shape[1]), columns=[fields])
        elif level == 'word':
            df.iloc[:, 3:] = [(getattr(df,field).values - getattr(df,field).values.min())/\
                              (getattr(df,field).values.max() - getattr(df,field).values.min())\
                              for field in fields[3:]]
    elif scaling == 'standard':
        if level == 'sentence':
            features = np.array([(feat - np.mean(feat))/np.std(feat) for feat in features.T])
            df = pd.DataFrame(data=features.T, index=range(features.shape[1]), columns=[fields])
        elif level == 'word':
            df.iloc[:, 3:] = [(getattr(df,field).values - getattr(df,field).values.mean())/\
                              getattr(df,field).values.std() for field in fields[3:]]
    return df

In [ ]:
#sbj3 = mk_dataframe('task1', 2, level='sentence', scaling='min-max')

In [4]:
# instantiate data transformer object for task 1 (NR) on sentence level with min-max feature scaling
datatransform = DataTransformer('task1', level='sentence', scaling='min-max')

In [ ]:
# get data for all subjects according to settings in cell above
# NOTE: data for each sbj is stored in a Pandas DataFrame
sbjs = [datatransform(0), datatransform(1), datatransform(2), 
        datatransform(3), datatransform(4), datatransform(5), 
        datatransform(6), datatransform(7),datatransform(8),
       datatransform(9), datatransform(10), datatransform(11)]